<a href="https://colab.research.google.com/github/acardenasor/MetNumUN2022I/blob/main/Week4IterativeMethodsForLinearSystemsGroup3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple iteration for systems of linear equations

First, generate a random diagonally dominant matrix, for testing.

In [38]:
import numpy as np
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

In [39]:
A

array([[15.192,  0.622,  0.438,  0.785,  0.78 ,  0.273,  0.276,  0.802,
         0.958,  0.876],
       [ 0.358, 15.501,  0.683,  0.713,  0.37 ,  0.561,  0.503,  0.014,
         0.773,  0.883],
       [ 0.365,  0.615, 15.075,  0.369,  0.933,  0.651,  0.397,  0.789,
         0.317,  0.568],
       [ 0.869,  0.436,  0.802, 15.144,  0.704,  0.705,  0.219,  0.925,
         0.442,  0.909],
       [ 0.06 ,  0.184,  0.047,  0.675, 15.595,  0.533,  0.043,  0.561,
         0.33 ,  0.503],
       [ 0.112,  0.607,  0.566,  0.007,  0.617, 15.912,  0.791,  0.992,
         0.959,  0.792],
       [ 0.285,  0.625,  0.478,  0.196,  0.382,  0.054, 15.452,  0.982,
         0.124,  0.119],
       [ 0.739,  0.587,  0.472,  0.107,  0.229,  0.9  ,  0.417, 15.536,
         0.006,  0.301],
       [ 0.437,  0.612,  0.918,  0.626,  0.706,  0.15 ,  0.746,  0.831,
        15.634,  0.438],
       [ 0.153,  0.568,  0.528,  0.951,  0.48 ,  0.503,  0.537,  0.819,
         0.057, 15.669]])

# I.  Jacobi iteration

Given

$$
A x = b
$$

separate the diagonal part $D$,

$$ A = D + (A - D) $$

and write

$$
x = D^{-1} (D - A) x + D^{-1} b\;.
$$

Then iterate

$$
x_{n + 1} = B x_{n} + c\;,
$$

where 

$$
B = D^{-1} (A - D) \qquad \text{and} \qquad c = D^{-1} b
$$


Let's construct the matrix and the r.h.s. for the Jacobi iteration

In [40]:
diag_1d = np.diag(A)

B = -A.copy()
np.fill_diagonal(B, 0)

D = np.diag(diag_1d)
invD = np.diag(1./diag_1d)
BB = invD @ B 
c = invD @ b

In [41]:
# sanity checks
from numpy.testing import assert_allclose

assert_allclose(-B + D, A)


# xx is a "ground truth" solution, compute it using a direct method
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@xx, b)
np.testing.assert_allclose(D@xx, B@xx + b)
np.testing.assert_allclose(xx, BB@xx + c)

Check that $\| B\| \leqslant 1$:

In [42]:
np.linalg.norm(BB)

0.36436161983015336

### Do the Jacobi iteration

In [43]:
n_iter = 50

x0 = np.ones(n)
x = x0
for _ in range(n_iter):
    x = BB @ x + c

In [44]:
# Check the result:

A @ x - b

array([ 0.,  0., -0., -0.,  0.,  0., -0.,  0., -0.,  0.])

### Task I.1

Collect the proof-of-concept above into a single function implementing the Jacobi iteration. This function should receive the r.h.s. matrix $A$, the l.h.s. vector `b`, and the number of iterations to perform.


The matrix $A$ in the illustration above is strongly diagonally dominant, by construction. 
What happens if the diagonal matrix elements of $A$ are made smaller? Check the convergence of the Jacobi iteration, and check the value of the norm of $B$.

(20% of the total grade)


In [45]:
def jacobi(A, b, num_it=1000):
    diag_A = np.diag(A)    
    BB = -A.copy()             
    np.fill_diagonal(BB, 0)    
    D = np.diag(diag_A)        
    invD = np.diag(1./diag_A)
    B = invD @ BB             
    c = invD @ b
    norm = np.linalg.norm(B)
    
    x0 = np.ones(A.shape[0])
    x = x0
    for _ in range(num_it):
        x = B @ x + c
    
    return x, norm

In [46]:
jacobi(A, b)

(array([ 0.039,  0.038,  0.043,  0.024,  0.057, -0.   , -0.006,  0.032,
        -0.004,  0.053]), 0.36436161983015336)

In [47]:
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15] * n)    # CHANGE THIS
b = rndm.uniform(size=n)

print(A)

[[15.192  0.622  0.438  0.785  0.78   0.273  0.276  0.802  0.958  0.876]
 [ 0.358 15.501  0.683  0.713  0.37   0.561  0.503  0.014  0.773  0.883]
 [ 0.365  0.615 15.075  0.369  0.933  0.651  0.397  0.789  0.317  0.568]
 [ 0.869  0.436  0.802 15.144  0.704  0.705  0.219  0.925  0.442  0.909]
 [ 0.06   0.184  0.047  0.675 15.595  0.533  0.043  0.561  0.33   0.503]
 [ 0.112  0.607  0.566  0.007  0.617 15.912  0.791  0.992  0.959  0.792]
 [ 0.285  0.625  0.478  0.196  0.382  0.054 15.452  0.982  0.124  0.119]
 [ 0.739  0.587  0.472  0.107  0.229  0.9    0.417 15.536  0.006  0.301]
 [ 0.437  0.612  0.918  0.626  0.706  0.15   0.746  0.831 15.634  0.438]
 [ 0.153  0.568  0.528  0.951  0.48   0.503  0.537  0.819  0.057 15.669]]


In [48]:
jacobi(A, b, 5000)

(array([ 0.039,  0.038,  0.043,  0.024,  0.057, -0.   , -0.006,  0.032,
        -0.004,  0.053]), 0.36436161983015336)

# II. Seidel's iteration.

##### Task II.1

Implement the Seidel's iteration. 

Test it on a random matrix. Study the convergence of iterations, relate to the norm of the iteration matrix.

(30% of the total grade)

In [49]:
def seidel(A, b, n_iter=1000):
    diag_A = np.diag(A)        
    D = np.diag(diag_A)        
    invD = np.diag(1./diag_A)
    U = np.triu(A) - D
    L = np.tril(A) - D
    B_tilde = -np.linalg.inv(D+L) @ U 
    c_tilde = np.linalg.inv(D+L) @ b 
    
    norm = np.linalg.norm(B)
    
    x0 = np.ones(A.shape[0])
    x = x0
    
    for _ in range(n_iter):
        x = B_tilde @ x + c_tilde
    
    return x, norm

In [50]:
np.set_printoptions(precision=3, suppress=True)

iters = range(1,15)

for i in iters:
    print('Jacobi', jacobi(A, b, i)[0])
    print('Seidel', seidel(A, b, i)[0])
    print('')

Jacobi [-0.332 -0.268 -0.279 -0.36  -0.126 -0.333 -0.208 -0.204 -0.342 -0.233]
Seidel [-0.332 -0.237 -0.196 -0.185 -0.052 -0.192 -0.051  0.064  0.02   0.096]

Jacobi [0.15  0.135 0.135 0.139 0.117 0.092 0.054 0.106 0.094 0.138]
Seidel [ 0.072  0.064  0.058  0.03   0.06  -0.006 -0.011  0.028 -0.008  0.051]

Jacobi [ 0.006  0.009  0.015 -0.011  0.04  -0.028 -0.024  0.01  -0.034  0.027]
Seidel [ 0.038  0.037  0.043  0.024  0.058  0.    -0.005  0.032 -0.004  0.053]

Jacobi [ 0.049  0.046  0.051  0.034  0.063  0.008 -0.     0.038  0.005  0.061]
Seidel [ 0.039  0.038  0.043  0.024  0.057 -0.    -0.006  0.032 -0.004  0.053]

Jacobi [ 0.036  0.035  0.04   0.021  0.056 -0.003 -0.007  0.03  -0.007  0.051]
Seidel [ 0.039  0.038  0.043  0.024  0.057 -0.    -0.006  0.032 -0.004  0.053]

Jacobi [ 0.04   0.039  0.044  0.025  0.058  0.    -0.005  0.032 -0.003  0.054]
Seidel [ 0.039  0.038  0.043  0.024  0.057 -0.    -0.006  0.032 -0.004  0.053]

Jacobi [ 0.039  0.038  0.043  0.023  0.057 -0.001 -0.006

# III. Minimum residual scheme

### Task III.1

Implement the $\textit{minimum residual}$ scheme: an explicit non-stationary method, where at each step you select the iteration parameter $\tau_n$ to minimize the residual $\mathbf{r}_{n+1}$ given $\mathbf{r}_n$. Test it on a random matrix, study the convergence to the solution, in terms of the norm of the residual and the deviation from the ground truth solution (which you can obtain using a direct method). Study how the iteration parameter $\tau_n$ changes as iterations progress.

(50% of the grade)

In [51]:
def minres(A, b, n_iter=500):
    x = np.ones(b.shape[0])
    T = []
    for _ in range(n_iter):
        r = A @ x - b
        tau = (r @ A @ r)/np.linalg.norm(A @ r)**2
        x = x - tau*r
        T.append(tau)
    return x, T

x = minres(A, b)[0]

np.testing.assert_allclose(A@x, b)
np.testing.assert_allclose(x, xx)

In [52]:
minres(A, b)[1]

[0.049458091915919704,
 0.06632815899696641,
 0.056465283719342306,
 0.05913686144833426,
 0.056149365850670686,
 0.05962056630692138,
 0.05532125962259336,
 0.06019139542967885,
 0.054726519624928915,
 0.060918354718292295,
 0.05478879426160098,
 0.061543172821534486,
 0.055177211118735095,
 0.06162125672274042,
 0.055145295102092665,
 0.0617011494677153,
 0.05472584159792917,
 0.062121454652208655,
 0.054002133608905656,
 0.0644186170983185,
 0.05443303564328217,
 0.06011645923749506,
 0.06271150857694172,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0